# Gender Classification using CNN

Представляю вашему вниманию мой первый опыт обучения нейронных сетей. Работа была проделана меньше чем за неделю без никакого бэкграунда в нейронных сетях, я только делал классический МЛ. Здесь я расскажу общую архитектуру модели, внизу же про проблемы с которыми столкнулся и как их решал, какие методы можно применить, чтобы решить оставшиеся.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2    
from sklearn.metrics import f1_score
import glob
from keras.preprocessing import image
from keras.models import Sequential, Model
from keras.utils import to_categorical
from keras.layers import Conv2D, MaxPooling2D,AveragePooling2D, Flatten, Dense, Dropout, Activation , Concatenate, Input , BatchNormalization, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator ,img_to_array, load_img
from keras.preprocessing import image
from tensorflow.keras.callbacks import EarlyStopping
from keras.optimizers import SGD
from keras.utils import plot_model
from keras import Model
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from sklearn.model_selection import train_test_split
%matplotlib inline

## Предобработка данных


In [2]:
df = pd.read_csv('../input/nikitapupu/data (1).csv')

In [3]:
nd = [0]*50000
nr = [1]*25001
ny = [2]*25001
nt = nd+nr+ny

In [4]:
datar = pd.DataFrame(nt, columns=['partition'])

In [5]:
df_par_attr = df.join(datar)

In [6]:
df_par_attr.set_index('path', inplace=True)

### Тут всего лишь вместо индекса делаю названия картинок, чтобы потом иметь доступ на их путь, маркирую под пол ('label') и делю на
partition
        0 -> train
        1 -> validation
        2 -> test

In [7]:
df_par_attr

,label,partition
path,,
male/085992.jpg,0,0
male/046332.jpg,0,0
female/097359.jpg,1,0
female/170442.jpg,1,0
male/008056.jpg,0,0
...,...,...
female/174616.jpg,1,2
female/177533.jpg,1,2
male/095538.jpg,0,2


### Предобработка данных: функция для перевода картинок в матрицы

In [8]:
images_folder = "../input/ntechdata/internship_data/"
def load_reshape_img(fname):
    img = load_img(fname, target_size = (224, 224), color_mode='rgb')
    x = img_to_array(img)/255.
    x = x.reshape((1,) + x.shape)

    return x

from keras.utils import np_utils

def generate_df(partition, attr, num_samples):
    '''
    partition
        0 -> train
        1 -> validation
        2 -> test
    
    '''
    
    df_ = df_par_attr[(df_par_attr['partition'] == partition) 
                           & (df_par_attr[attr] == 0)].sample(int(num_samples/2))
    df_ = pd.concat([df_,
                      df_par_attr[(df_par_attr['partition'] == partition) 
                                  & (df_par_attr[attr] == 1)].sample(int(num_samples/2))])

    # for Train and Validation
    if partition != 2:
        x_ = np.array([load_reshape_img(images_folder + fname) for fname in df_.index])
        x_ = x_.reshape(x_.shape[0], 224, 224, 3)
        y_ = np_utils.to_categorical(df_[attr],2)
    # for Test
    else:
        x_ = []
        y_ = []

        for index, target in df_.iterrows():
            im = cv2.imread(images_folder + index)
            im = cv2.resize(cv2.cvtColor(im, cv2.COLOR_BGR2RGB), (IMG_WIDTH, IMG_HEIGHT)).astype(np.float32) / 255.0
            im = np.expand_dims(im, axis =0)
            x_.append(im)
            y_.append(target[attr])

    return x_, y_

In [9]:
TRAINING_SAMPLES = 10000
VALIDATION_SAMPLES = 2500
TEST_SAMPLES = 2500
IMG_WIDTH = 224
IMG_HEIGHT = 224
BATCH_SIZE = 16
NUM_EPOCHS = 4

### разделяю данные на test, train

In [10]:
from keras.applications.resnet50 import ResNet50, preprocess_input
x_train, y_train = generate_df(0, 'label', TRAINING_SAMPLES)
images_folder = "../input/ntechdata/internship_data/"
# Train - Data Preparation - Data Augmentation with generators
train_datagen =  ImageDataGenerator(
  preprocessing_function=preprocess_input,
  rotation_range=30,
  width_shift_range=0.2,
  height_shift_range=0.2,
  shear_range=0.2,
  zoom_range=0.2,
  horizontal_flip=True,
)

train_datagen.fit(x_train)

train_generator = train_datagen.flow(
x_train, y_train,
batch_size=BATCH_SIZE,
)

In [11]:
x_valid, y_valid = generate_df(1, 'label', VALIDATION_SAMPLES)

valid_datagen =  ImageDataGenerator(
  preprocessing_function=preprocess_input,
  rotation_range=30,
  width_shift_range=0.2,
  height_shift_range=0.2,
  shear_range=0.2,
  zoom_range=0.2,
  horizontal_flip=True,
)

valid_datagen.fit(x_valid)

valid_generator = valid_datagen.flow(
x_valid, y_valid,
batch_size=BATCH_SIZE,
)

## архитектура модели - ResNet50 + предобученные веса
### большая проблема при выполнении тестового была в том, что невозможно засунуть все 100к картинок в один массив np.array, поэтому я делал это сэмплами по 10к. То есть брал первые 10к,сохранял веса и на этих весах новые 10к и так 5-6 раз. 
### из-за такой архитектуры решения на последний день тяжело построить скрипт, нужный для тестового решения, который бы выдавал json файл. Только под конец я понял, что нужно строить функцию, которая может предсказывать дискретно, а не сэмплами по 10к, но было уже поздно. Первый опыт, который научил меня заранее продумывать архитектуру решения под задание, а не просто использовать best practices на каггле.

In [12]:
inc_model = ResNet50(weights='imagenet',
                        include_top=False,
                        input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))

print("number of layers:", len(inc_model.layers))
#inc_model.summary()

94773248/94765736 [==============================] - 2s 0us/step
number of layers: 175


In [13]:
#Adding custom Layers
x = inc_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(512, activation="relu")(x)
predictions = Dense(2, activation="softmax")(x)

In [14]:
model_ = Model(inputs=inc_model.input, outputs=predictions)

In [15]:
model_.load_weights('../input/jhlsfg/Scurt_best_1.hdf5')

In [16]:
# creating the final model 




# Lock initial layers to do not be trained
for layer in model_.layers[:52]:
    layer.trainable = False

# compile the model

model_.compile(optimizer=SGD(lr=0.0001, momentum=0.9)
                    , loss='binary_crossentropy'
                    , metrics=['accuracy'])

In [17]:
checkpoint_path = "../input/durtuso"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [18]:
from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath='orty.best.hdf5', 
                               verbose=1, save_weights_only=True, period = 1)

# Обучаем

In [ ]:
hist = model_.fit_generator(train_generator
                     , validation_data = valid_generator
                      , steps_per_epoch= TRAINING_SAMPLES/BATCH_SIZE
                      , epochs= NUM_EPOCHS
                      , callbacks=[checkpointer]
                      , verbose=1
                            
                    )

Epoch 1/4
625/625 [==============================] - ETA: 0s - loss: 0.2021 - accuracy: 0.9191
Epoch 00001: saving model to orty.best.hdf5
625/625 [==============================] - 2218s 4s/step - loss: 0.2021 - accuracy: 0.9191 - val_loss: 0.1723 - val_accuracy: 0.9328
Epoch 2/4
 11/625 [..............................] - ETA: 31:11 - loss: 0.1812 - accuracy: 0.9205

## Как улучшить модель?
Итак, модель обучается с метрикой **в 93.3%**
- использовать все 100к картинок с помощью мЕньших сэмплов. Например, можно сделать сэмплы по 100 картинок и так пройти весь датасет. 
- экспериментировать со слоями - я мало этого делал, так как скорость моего программирования была очень маленькой из-за того, что я сталкиваюсь с такими вещами впервые
- увеличить датасет с помощью augmentation


## Почему я не успел сделать скрипт?
Как я уже говорил, это моя первая нейронная сетка и меньше чем за неделю я столнулся с большим количеством проблем. Начиная с неправильной предобработкой в ImageDataGenerator из-за разного разрешения картинок, заканчивая не изменяющимся val_accuracy в процессе обучения из-за неправильной нормализации valid данных. Только в последние 3 часа я решил все проблемы и остался один на один с задачей построения скрипта в условиях усталости после **недельного спринта изучения tensorflow**. Чтобы успешно сделать скрипт, мне нужно поменять функцию generate_df так,чтобы можно было делать не сэмплы по 10к, но любое заданное количество картинок, на что я уже неспособен из-за усталости

## Conclusion
Для меня сейчас начинается самая кайфовая часть - понять, что я вообще сделал с теоретической точки зрения? 
Это я собираюсь сделать с помощью Deep Learning Illustrated by Krohn,и несмотря на любой результат тестового, я явно научился многому, в ситуации без дедлайна я бы не успел сделать так много